In [21]:
from transformers import pipeline

In [22]:
# Cargar pipeline de análisis de sentimiento
sentiment_analysis = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [23]:
import pandas as pd

# Cargamos el dataset
df = pd.read_csv('raw/fichajes_news.csv', sep=';')
df.head()

,title,date,content
0,El Real Madrid lo tiene claro: negativa al AC ...,01/08,El Real Madrid ha rechazado un acercamiento po...
1,La Atalanta estudia moverse por una pieza de l...,01/08,Gian Piero Gasperini confía en potenciar toda...
2,La Juventus estudia un fichaje de 20 M€,01/08,Decidida a mejorar las prestaciones de su ataq...
3,Osasuna confirma la venta de David García,01/08,Se venía apuntando a esta posibilidad desde h...
4,El Atlético busca la fórmula para fichar a Al...,01/08,Aunque Alexander Sörloth ya ha dado el sí qui...


In [24]:
df['whole'] = df['title'] + ' ' + df['content']
df['whole'] = df['whole'].astype(str)

# Dividimos en chunks de 256 palabras
def chunk_text(text, chunk_size=256):
    chunks = []
    for i in range(0, len(text), chunk_size):
        chunks.append(text[i:i+chunk_size])
    return chunks

df['chunks'] = df['whole'].apply(chunk_text)
df.head()

,title,date,content,whole,chunks
0,El Real Madrid lo tiene claro: negativa al AC ...,01/08,El Real Madrid ha rechazado un acercamiento po...,El Real Madrid lo tiene claro: negativa al AC ...,[El Real Madrid lo tiene claro: negativa al AC...
1,La Atalanta estudia moverse por una pieza de l...,01/08,Gian Piero Gasperini confía en potenciar toda...,La Atalanta estudia moverse por una pieza de l...,[La Atalanta estudia moverse por una pieza de ...
2,La Juventus estudia un fichaje de 20 M€,01/08,Decidida a mejorar las prestaciones de su ataq...,La Juventus estudia un fichaje de 20 M€ Decidi...,[La Juventus estudia un fichaje de 20 M€ Decid...
3,Osasuna confirma la venta de David García,01/08,Se venía apuntando a esta posibilidad desde h...,Osasuna confirma la venta de David García Se ...,[Osasuna confirma la venta de David García Se...
4,El Atlético busca la fórmula para fichar a Al...,01/08,Aunque Alexander Sörloth ya ha dado el sí qui...,El Atlético busca la fórmula para fichar a Al...,[ El Atlético busca la fórmula para fichar a A...


In [25]:
# Analizamos el sentimiento de cada chunk
df['sentiment'] = df['chunks'].apply(sentiment_analysis)

def score(sentiments):
    return sum([s['score'] if s['label'] == 'POSITIVE' else -s['score'] for s in sentiments]) / len(sentiments)

df['score'] = df['sentiment'].apply(score)
df.head()

,title,date,content,whole,chunks,sentiment,score
0,El Real Madrid lo tiene claro: negativa al AC ...,01/08,El Real Madrid ha rechazado un acercamiento po...,El Real Madrid lo tiene claro: negativa al AC ...,[El Real Madrid lo tiene claro: negativa al AC...,"[{'label': 'NEGATIVE', 'score': 0.957447886466...",-0.733367
1,La Atalanta estudia moverse por una pieza de l...,01/08,Gian Piero Gasperini confía en potenciar toda...,La Atalanta estudia moverse por una pieza de l...,[La Atalanta estudia moverse por una pieza de ...,"[{'label': 'NEGATIVE', 'score': 0.868134260177...",-0.914856
2,La Juventus estudia un fichaje de 20 M€,01/08,Decidida a mejorar las prestaciones de su ataq...,La Juventus estudia un fichaje de 20 M€ Decidi...,[La Juventus estudia un fichaje de 20 M€ Decid...,"[{'label': 'NEGATIVE', 'score': 0.982062876224...",-0.659285
3,Osasuna confirma la venta de David García,01/08,Se venía apuntando a esta posibilidad desde h...,Osasuna confirma la venta de David García Se ...,[Osasuna confirma la venta de David García Se...,"[{'label': 'NEGATIVE', 'score': 0.992787539958...",-0.932613
4,El Atlético busca la fórmula para fichar a Al...,01/08,Aunque Alexander Sörloth ya ha dado el sí qui...,El Atlético busca la fórmula para fichar a Al...,[ El Atlético busca la fórmula para fichar a A...,"[{'label': 'NEGATIVE', 'score': 0.970461726188...",-0.688004


In [27]:
# Añaadimos una columna con el sentimiento global
df['global_sentiment'] = df['score'].apply(lambda x: 'POSITIVE' if x > 0 else 'NEGATIVE')

df['global_sentiment'].value_counts()

global_sentiment
NEGATIVE    15325
POSITIVE      270
Name: count, dtype: int64

In [28]:
# Guardamos el resultado
df.to_csv('raw/fichajes_news.csv', sep=';', index=False)